<a href="https://colab.research.google.com/github/ucaokylong/agents_stuff/blob/main/AI_Search_Agents.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Yêu cầu:
- [OpenAI key](https://platform.openai.com/settings/organization/api-keys) để làm backend LLMs
- [Serp API](https://serper.dev/api-key) key để tìm trên Google

In [ ]:
!pip install serpapi
!pip install google-search-results

  Preparing metadata (setup.py) ... done
  Created wheel for google-search-results: filename=google_search_results-2.4.2-py3-none-any.whl size=32009 sha256=612c4aa8481f0d3c57ad9dc5fbd79e602e27900f252e639d08f84e3b7cb35a9a
  Stored in directory: /root/.cache/pip/wheels/6e/42/3e/aeb691b02cb7175ec70e2da04b5658d4739d2b41e5f73cd06f
Successfully built google-search-results


In [ ]:
from openai import OpenAI
from google.colab import userdata

def reasoning_step(state, user_input, intermediate_results):
    client = OpenAI(api_key=userdata.get('open_ai_key'))

    # Construct the assistant message dynamically with the state, user input, and intermediate results
    messages = [
        {
            "role": "system",
            "content": (
                "You are a reasoning and acting agent. Based on the current state and user input, decide the next action.\n"
                f"State: {state}\n"
                f"User Input: {user_input}\n"
                f"Intermediate Results: {intermediate_results}\n\n"
                "Respond with one of these actions:\n"
                "- Search(query)\n"
                "- Do nothing\n"
                "- Summarize(results)\n"
            )
        }
    ]

    response = client.chat.completions.create(
        model="gpt-4o",
        messages=messages,
        response_format={
            "type": "json_schema",
            "json_schema": {
                "name": "agent_schema",
                "schema": {
                    "type": "object",
                    "required": [],
                    "properties": {}
                },
                "strict": False
            }
        },
        tools=[
            {
                "type": "function",
                "function": {
                    "name": "search_action",
                    "strict": True,
                    "parameters": {
                        "type": "object",
                        "required": ["query"],
                        "properties": {
                            "query": {
                                "type": "string",
                                "description": "The search query string to be submitted to the search engine."
                            }
                        },
                        "additionalProperties": False
                    },
                    "description": "Performs a search using the SERP API and returns the organic results."
                }
            },
            {
                "type": "function",
                "function": {
                    "name": "do_nothing",
                    "strict": True,
                    "parameters": {
                        "type": "object",
                        "required": [],
                        "properties": {},
                        "additionalProperties": False
                    },
                    "description": "A function that performs no action."
                }
            },
            {
                "type": "function",
                "function": {
                    "name": "summarize_action",
                    "description": "Summarizes product results using OpenAI's GPT-4 model",
                    "parameters": {
                    "type": "object",
                    "required": [
                        "results"
                    ],
                    "properties": {
                        "results": {
                        "type": "string",
                        "description": "The product results to summarize"
                        }
                    },
                    "additionalProperties": False
                    },
                    "strict": True
                }
            }
        ],
        tool_choice="required",
        temperature=1,
        max_completion_tokens=2048,
        top_p=1,
        frequency_penalty=0,
        presence_penalty=0
    )

    return response

Giả sử chưa search bao giờ:

In [ ]:
assistant_message = reasoning_step(
    "start",
    "iphone 14 price?",
    []
).choices[0].message

In [ ]:
assistant_message

ChatCompletionMessage(content=None, refusal=None, role='assistant', audio=None, function_call=None, tool_calls=[ChatCompletionMessageToolCall(id='call_TrusIexZ45jCThW8TlLu8vuw', function=Function(arguments='{"query":"iPhone 14 price"}', name='search_action'), type='function')], annotations=[])

Đã search và có thông tin rồi

In [ ]:
assistant_message = reasoning_step(
    "start",
    "iphone 14 price?",
    [{
            "action": "search_action",
            "parameters": "iPhone 14 price",
            "result": "iPhone 15 is $1000"
    }
     ]
).choices[0].message

In [ ]:
assistant_message

ChatCompletionMessage(content=None, refusal=None, role='assistant', audio=None, function_call=None, tool_calls=[ChatCompletionMessageToolCall(id='call_awKDh75nSER0AjPFWEmSo93l', function=Function(arguments='{"query": "iPhone 14 price"}', name='search_action'), type='function'), ChatCompletionMessageToolCall(id='call_BHWS6yOP5KMMVykyo61M88Q0', function=Function(arguments='{"results": "iPhone 15 is $1000"}', name='summarize_action'), type='function')], annotations=[])

Đánh giá chất lượng


```
Query: iphone 14 price?
Expect: trong output cuối cùng có giá $500

```




In [ ]:
def process_tool_calls(response):
    # Extract tool call information from the response
    tool_calls = response.tool_calls

    if tool_calls:
        # Extract tool function details
        tool_call_id = tool_calls[0].id
        tool_function_name = tool_calls[0].function.name
        tool_query_string = eval(tool_calls[0].function.arguments)

        # Print extracted details
        print('Tool Function Name:', tool_function_name)
        print('Tool Query String:', tool_query_string)

        return tool_function_name, tool_query_string

    else:
        print("No tool calls identified.")
        return None, None

process_tool_calls(assistant_message)

Tool Function Name: search_action
Tool Query String: {'query': 'iPhone 14 price'}


('search_action', {'query': 'iPhone 14 price'})

In [ ]:
def search_action(query):
    from serpapi import GoogleSearch
    api_key = userdata.get("serpapi")
    search = GoogleSearch({"q": query, "api_key": api_key, "num": 10})
    results = search.get_dict()

    info = []

    print('--->', results)

    for item in results['organic_results']:
        info.append(item['snippet'])
    return "\n".join(info)

In [ ]:
search_action("Thời tiết hôm nay Hà Nội đi bơi ở Hồ Gươm")

---> {'search_metadata': {'id': '67dbd7e938479b69a3934d48', 'status': 'Success', 'json_endpoint': 'https://serpapi.com/searches/166e6e3cf41e4bbb/67dbd7e938479b69a3934d48.json', 'created_at': '2025-03-20 08:55:05 UTC', 'processed_at': '2025-03-20 08:55:05 UTC', 'google_url': 'https://www.google.com/search?q=Th%E1%BB%9Di+ti%E1%BA%BFt+h%C3%B4m+nay+H%C3%A0+N%E1%BB%99i+%C4%91i+b%C6%A1i+%E1%BB%9F+H%E1%BB%93+G%C6%B0%C6%A1m&oq=Th%E1%BB%9Di+ti%E1%BA%BFt+h%C3%B4m+nay+H%C3%A0+N%E1%BB%99i+%C4%91i+b%C6%A1i+%E1%BB%9F+H%E1%BB%93+G%C6%B0%C6%A1m&num=10&sourceid=chrome&ie=UTF-8', 'raw_html_file': 'https://serpapi.com/searches/166e6e3cf41e4bbb/67dbd7e938479b69a3934d48.html', 'total_time_taken': 1.4}, 'search_parameters': {'engine': 'google', 'q': 'Thời tiết hôm nay Hà Nội đi bơi ở Hồ Gươm', 'google_domain': 'google.com', 'num': '10', 'device': 'desktop'}, 'search_information': {'query_displayed': 'Thời tiết hôm nay Hà Nội đi bơi ở Hồ Gươm', 'total_results': 59800, 'time_taken_displayed': 0.32, 'organic_r

'Thời tiết hiện tại ; RealFeel®. 62° ; Gió. B 5 mi/h ; Gió giật mạnh. 10 mi/h ; Độ ẩm. 58% ; Điểm sương. 48° F.\nThời tiết; Thời tiết 10 ngày tới. Thời tiết Hà Nội. Thời tiết hiện tại. Cập nhật: 7h 20/03/2025. Nhiệt độ. : 18°C. Thời tiết. : Nhiều mây, không mưa. Độ ẩm.\nRealFeel Shade™57°. GióTB 7 mi/h. Chất lượng không khíRất có hại. Chỉ số UV tối đa1 Thấp. Gió giật7 mi/h. Độ ẩm67%. Điểm sương48° F. Mật độ mây0%.\nTiếp xúc quá mức với ánh sáng mặt trời trong thời gian ngắn sẽ gây bỏng nắng, tổn thương mắt như đục thủy tinh thể, da bị bỏng, khô, sạm, tạo nếp nhăn, lão hóa ...\nThời tiết Quận Hoàn Kiếm - Hà Nội những ngày tới ; Hôm nay. Nhiều mây. 53 % · 16.4 °C · /. 18.4 °C ; T4 Ngày 19. Nhiều mây. 65 % · 15.8 °C · /. 21.9 °C ; T5 Ngày 20.\n16 ° / 16 °. Mây đen u ám. 3 %. Mây đen u ám. Thời tiết 48h. Thời tiết Hoàn Kiếm - Hà Nội những ngày tới. Hiện tại. 14° / 19°. Mưa nhẹ. 100 %. 18.83 km/h. Ngày/ ...\nIn Hoàn Kiếm, mùa ẩm ướt thì nóng, ngột ngạt và mây bao phủ và mùa khô thì thoải má

In [ ]:
def do_nothing():
    return None

In [ ]:
def summarize_action(results):
    client = OpenAI(api_key=userdata.get('open_ai_key'))
    prompt = f"Summarize the following product results:\n{results}"

    response = client.chat.completions.create(
        model="gpt-4o",
        messages=[
            {"role": "user", "content": prompt},
        ]
    )
    return response.choices[0].message.content


In [ ]:
def answer_question(question, search_results):
    client = OpenAI(api_key=userdata.get('open_ai_key'))
    prompt = f"Answer the following question using the provided search results:\n\nQuestion: {question}\n\nSearch Results:\n{search_results}"
    response = client.chat.completions.create(
        model="gpt-4o",
        messages=[
            {"role": "user", "content": prompt},
        ]
    )
    return response.choices[0].message.content


In [ ]:
function_mapping = {
    "search_action": search_action,
    "do_nothing": do_nothing,
    "summarize_action": summarize_action,
    "answer_question": answer_question,
}



In [ ]:
function_mapping["search_action"]("Thời tiết hôm nay Hà Nội đi bơi ở Hồ Gươm")

---> {'search_metadata': {'id': '67dbd7e938479b69a3934d48', 'status': 'Success', 'json_endpoint': 'https://serpapi.com/searches/166e6e3cf41e4bbb/67dbd7e938479b69a3934d48.json', 'created_at': '2025-03-20 08:55:05 UTC', 'processed_at': '2025-03-20 08:55:05 UTC', 'google_url': 'https://www.google.com/search?q=Th%E1%BB%9Di+ti%E1%BA%BFt+h%C3%B4m+nay+H%C3%A0+N%E1%BB%99i+%C4%91i+b%C6%A1i+%E1%BB%9F+H%E1%BB%93+G%C6%B0%C6%A1m&oq=Th%E1%BB%9Di+ti%E1%BA%BFt+h%C3%B4m+nay+H%C3%A0+N%E1%BB%99i+%C4%91i+b%C6%A1i+%E1%BB%9F+H%E1%BB%93+G%C6%B0%C6%A1m&num=10&sourceid=chrome&ie=UTF-8', 'raw_html_file': 'https://serpapi.com/searches/166e6e3cf41e4bbb/67dbd7e938479b69a3934d48.html', 'total_time_taken': 1.4}, 'search_parameters': {'engine': 'google', 'q': 'Thời tiết hôm nay Hà Nội đi bơi ở Hồ Gươm', 'google_domain': 'google.com', 'num': '10', 'device': 'desktop'}, 'search_information': {'query_displayed': 'Thời tiết hôm nay Hà Nội đi bơi ở Hồ Gươm', 'total_results': 59800, 'time_taken_displayed': 0.32, 'organic_r

'Thời tiết hiện tại ; RealFeel®. 62° ; Gió. B 5 mi/h ; Gió giật mạnh. 10 mi/h ; Độ ẩm. 58% ; Điểm sương. 48° F.\nThời tiết; Thời tiết 10 ngày tới. Thời tiết Hà Nội. Thời tiết hiện tại. Cập nhật: 7h 20/03/2025. Nhiệt độ. : 18°C. Thời tiết. : Nhiều mây, không mưa. Độ ẩm.\nRealFeel Shade™57°. GióTB 7 mi/h. Chất lượng không khíRất có hại. Chỉ số UV tối đa1 Thấp. Gió giật7 mi/h. Độ ẩm67%. Điểm sương48° F. Mật độ mây0%.\nTiếp xúc quá mức với ánh sáng mặt trời trong thời gian ngắn sẽ gây bỏng nắng, tổn thương mắt như đục thủy tinh thể, da bị bỏng, khô, sạm, tạo nếp nhăn, lão hóa ...\nThời tiết Quận Hoàn Kiếm - Hà Nội những ngày tới ; Hôm nay. Nhiều mây. 53 % · 16.4 °C · /. 18.4 °C ; T4 Ngày 19. Nhiều mây. 65 % · 15.8 °C · /. 21.9 °C ; T5 Ngày 20.\n16 ° / 16 °. Mây đen u ám. 3 %. Mây đen u ám. Thời tiết 48h. Thời tiết Hoàn Kiếm - Hà Nội những ngày tới. Hiện tại. 14° / 19°. Mưa nhẹ. 100 %. 18.83 km/h. Ngày/ ...\nIn Hoàn Kiếm, mùa ẩm ướt thì nóng, ngột ngạt và mây bao phủ và mùa khô thì thoải má

In [ ]:
# function_to_call = function_mapping[tool_function_name]
# function_response = function_to_call(**tool_query_string)

In [ ]:
from openai import OpenAI
from google.colab import userdata

def reasoning_step_advanced(state, user_input, intermediate_results):
    client = OpenAI(api_key=userdata.get('open_ai_key'))

    # Construct the assistant message dynamically with the state, user input, and intermediate results
    messages = [
        {
            "role": "system",
            "content": (
                "You are a reasoning and acting agent. Based on the current state and user input, decide the next action.\n"
                f"State: {state}\n"
                f"User Input: {user_input}\n"
                f"Intermediate Results: {intermediate_results}\n\n"
                "Respond with one of these actions:\n"
                "- Search(query)\n"
                "- Do nothing\n"
                "- answer_question(results)\n"
            )
        }
    ]

    response = client.chat.completions.create(
        model="gpt-4o",
        messages=messages,
        response_format={
            "type": "json_schema",
            "json_schema": {
                "name": "agent_schema",
                "schema": {
                    "type": "object",
                    "required": [],
                    "properties": {}
                },
                "strict": False
            }
        },
        tools=[
            {
                "type": "function",
                "function": {
                    "name": "search_action",
                    "strict": True,
                    "parameters": {
                        "type": "object",
                        "required": ["query"],
                        "properties": {
                            "query": {
                                "type": "string",
                                "description": "The search query string to be submitted to the search engine."
                            }
                        },
                        "additionalProperties": False
                    },
                    "description": "Performs a search using the SERP API and returns the organic results."
                }
            },
            {
                "type": "function",
                "function": {
                    "name": "do_nothing",
                    "strict": True,
                    "parameters": {
                        "type": "object",
                        "required": [],
                        "properties": {},
                        "additionalProperties": False
                    },
                    "description": "A function that performs no action."
                }
            },
            {
                "type": "function",
                "function": {
                    "name": "summarize_action",
                    "description": "Summarizes product results using OpenAI's GPT-4 model",
                    "parameters": {
                    "type": "object",
                    "required": [
                        "results"
                    ],
                    "properties": {
                        "results": {
                        "type": "string",
                        "description": "The product results to summarize"
                        }
                    },
                    "additionalProperties": False
                    },
                    "strict": True
                }
            },
            {
                "type": "function",
                "function": {
                    "name": "answer_question",
                    "description": "Answer question using the search result",
                    "parameters": {
                    "type": "object",
                    "required": [
                        "question",
                        "search_results"
                    ],
                    "properties": {
                        "question": {
                            "type": "string",
                            "description": "The question to be answered"
                        },
                        "search_results": {
                            "type": "string",
                            "description": "The search results from api to answer the question"
                        }
                    },
                    "additionalProperties": False
                    },
                    "strict": True
                }
            }
        ],
        tool_choice="required",
        temperature=1,
        max_completion_tokens=2048,
        top_p=1,
        frequency_penalty=0,
        presence_penalty=0
    )

    return response

In [ ]:
def react_agent(user_input):
    state = {"status": "start", "history": []}  # Initialize the state with status and history
    intermediate_results = []

    while True:
        # Reasoning step: Decide the next action
        action_response = reasoning_step_advanced(state, user_input, intermediate_results).choices[0].message

        action_response = process_tool_calls(action_response)
        tool_function_name, tool_query_string = action_response

        if tool_function_name == "do_nothing":
            return intermediate_results[-1]["result"]
            break

        # Execute the selected action
        function_to_call = function_mapping[tool_function_name]
        action_result = function_to_call(**tool_query_string)

        print(action_result)

        # Save intermediate results
        intermediate_results.append({
            "action": tool_function_name,
            "parameters": tool_query_string,
            "result": action_result
        })

        # Update state
        state.update({
            "status": "in_progress",
            "last_action": tool_function_name,
            "last_parameters": tool_query_string
            })

    return intermediate_results

user_query = "Thời tiết hôm nay có phù hợp để đi bơi ở Hồ Gươm không?"
final_answer = react_agent(user_query)
print(f"Final Output: {final_answer}")

Tool Function Name: search_action
Tool Query String: {'query': 'Thời tiết hôm nay ở Hồ Gươm'}
---> {'search_metadata': {'id': '67dbd800aff97e1631cef3ea', 'status': 'Success', 'json_endpoint': 'https://serpapi.com/searches/95549bff3dffbc86/67dbd800aff97e1631cef3ea.json', 'created_at': '2025-03-20 08:55:28 UTC', 'processed_at': '2025-03-20 08:55:28 UTC', 'google_url': 'https://www.google.com/search?q=Th%E1%BB%9Di+ti%E1%BA%BFt+h%C3%B4m+nay+%E1%BB%9F+H%E1%BB%93+G%C6%B0%C6%A1m&oq=Th%E1%BB%9Di+ti%E1%BA%BFt+h%C3%B4m+nay+%E1%BB%9F+H%E1%BB%93+G%C6%B0%C6%A1m&num=10&sourceid=chrome&ie=UTF-8', 'raw_html_file': 'https://serpapi.com/searches/95549bff3dffbc86/67dbd800aff97e1631cef3ea.html', 'total_time_taken': 1.71}, 'search_parameters': {'engine': 'google', 'q': 'Thời tiết hôm nay ở Hồ Gươm', 'google_domain': 'google.com', 'num': '10', 'device': 'desktop'}, 'search_information': {'query_displayed': 'Thời tiết hôm nay ở Hồ Gươm', 'total_results': 875000, 'time_taken_displayed': 0.28, 'organic_results